In [1]:
import unidecode
import string
import random
import math

import torch
from torch.utils.data import Dataset
from torchvision import datasets
import torchvision.transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

In [2]:
all_characters[10]

NameError: name 'all_characters' is not defined

In [3]:
# Reading and un-unicode-encoding data

all_characters = string.printable
n_characters = len(all_characters)

# This function reads the shakespear.txt file into jupyter, it also places \n where theres a newline and so on.
def read_file(filename):
    file = unidecode.unidecode(open(filename).read())
    return file, len(file)

# Turning a string into a tensor
# Turns every character into a tensor number corresponding to its position in the embedding.
def char_tensor(string):
    tensor = np.zeros(len(string))
    for c in range(len(string)):
        try:
            tensor[c] = all_characters.index(string[c])
        except:
            continue
    return tensor

def int_to_char(tensor):
    #string = np.zeros(len(tensor))
    #for i in range(len(tensor)):
    string = all_characters[tensor]
    return string
    
    

def random_training_set(chunk_len, batch_size):
    inp = torch.LongTensor(batch_size, chunk_len)
    target = torch.LongTensor(batch_size, chunk_len)
    for bi in range(batch_size):
        start_index = random.randint(0, file_len - chunk_len)
        end_index = start_index + chunk_len + 1
        chunk = file[start_index:end_index]
        inp[bi] = char_tensor(chunk[:-1])
        target[bi] = char_tensor(chunk[1:])
    inp = Variable(inp)
    target = Variable(target)
#     if args.cuda:
#         inp = inp.cuda()
#         target = target.cuda()
    return inp, target



In [4]:
# Convert all data to numbers.
file, file_len = read_file('shakespear.txt')
data_id = char_tensor(file)

In [5]:
# Creating the labels, which is the text but embedded and shifted one.
# The character after the input is the label.
to_onehot = nn.Embedding(1, n_characters) 
to_onehot.weight.data = torch.eye(n_characters)

In [6]:
from torch.autograd import Variable 


class LSTM1(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM1, self).__init__()
        self.num_classes = num_classes #nr of classes
        self.num_layers = num_layers #nr of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.dropout = nn.Dropout(0.3)

        self.lstm = nn.LSTM(input_size=input_size, hidden_size = hidden_size, num_layers = num_layers, batch_first=True) # LSTM layer
        self.fc_1 =  nn.Linear(hidden_size,hidden_size) # First FC layer
        self.fc = nn.Linear(hidden_size,num_classes) # Final FC layer

        self.relu = nn.ReLU()
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers,x.size(0),self.hidden_size, device=x.device)) # hidden state
        c_0 = Variable(torch.zeros(self.num_layers,x.size(0),self.hidden_size, device=x.device)) # internal state

        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state

        hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        out = self.relu(output)
        out = out[:, -1, :]
        out = self.fc_1(out) #first Dense
        out = self.relu(out) #relu
        out = self.fc(self.dropout(out)) #Final Output
        return out


num_epochs = 30 #2 epochs
learning_rate = 0.001 #0.001 lr
input_size = 1 #number of features
hidden_size = 256 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers
num_classes = n_characters
device=torch.device('cuda')

model = LSTM1(num_classes=num_classes, input_size=input_size, hidden_size=hidden_size, 
              num_layers=num_layers) #our lstm class
model.to(device)
# The inputs will be of size: (batch_nr, batch_size, embedding size) .

LSTM1(
  (dropout): Dropout(p=0.3, inplace=False)
  (lstm): LSTM(1, 256, batch_first=True)
  (fc_1): Linear(in_features=256, out_features=256, bias=True)
  (fc): Linear(in_features=256, out_features=100, bias=True)
  (relu): ReLU()
)

In [7]:
data_id = char_tensor(file)
seq_length = 100
dataX = []
dataY = []
for i in range(0, file_len - seq_length, 1):
    seq_in = data_id[i:i + seq_length]
    seq_out = data_id[i + seq_length]
    dataX.append(seq_in)
    dataY.append(seq_out)
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
X = torch.tensor(dataX, dtype=torch.float64).reshape(n_patterns, seq_length, 1)
X = X / float(n_characters)
y = torch.tensor(dataY)
print(X.shape, y.shape)


Total Patterns:  1115294


C:\Users\Olle\AppData\Local\Temp\ipykernel_16500\3002205204.py:12: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:233.)
  X = torch.tensor(dataX, dtype=torch.float64).reshape(n_patterns, seq_length, 1)


torch.Size([1115294, 100, 1]) torch.Size([1115294])


In [8]:
from torch.utils.data import Dataset
from torchvision import datasets
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
BATCH_SIZE = 128
loader =  DataLoader(TensorDataset(X, y), shuffle=True, batch_size=BATCH_SIZE)

In [92]:
len(loader)
torch.cuda.empty_cache() 

# Task 1

In [154]:

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss(reduction="sum") 
nr_of_batches = len(loader)
best_model = None
best_loss = np.inf

for epochs in range(num_epochs):
    model.train()
    loss_epoch = 0
    count = 0
    for X_batch, y_batch in loader:
        X_batch = X_batch.to(device=device)
        y_batch = y_batch.to(device=device)
        prediction = model(X_batch.float()) 
        loss = loss_function(prediction,y_batch.long())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        loss_epoch += torch.Tensor.cpu(loss).detach().numpy()
        count+=1
        print(
        f'\rEpoch % {round(float(count/nr_of_batches*100),2)} - Loss: {loss_epoch}',end='')
    model.eval()    
    loss_e = 0
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch = X_batch.to(device=device)
            y_batch = y_batch.to(device=device)
            y_pred = model(X_batch.float()) 
            loss_e += torch.Tensor.cpu(loss_function(y_pred, y_batch.long())).detach().numpy()
        if loss_e < best_loss:
            best_loss = loss_e
            best_model = model.state_dict()
        print()
        print("Epoch %d: Validation Loss: %f" % (epochs, loss_e))
 





Epoch % 100.0 - Loss: 2020729.6110877998Epoch 0: Cross-entropy: 1886346.7019
Epoch % 100.0 - Loss: 2005446.4873161316Epoch 1: Cross-entropy: 1865090.4125
Epoch % 100.0 - Loss: 2003583.4092292786Epoch 2: Cross-entropy: 1863455.5381
Epoch % 100.0 - Loss: 1996046.9385643005Epoch 3: Cross-entropy: 1851456.6763
Epoch % 53.98 - Loss: 1071252.1349334717

KeyboardInterrupt: 

# Task 2

In [138]:
torch.save(best_model, "LSTM_shakespear_best.pth")
torch.save(model, "LSTM_shakespear.pth")

In [26]:
seq_length = 5
start = np.random.randint(0, len(file)-seq_length)
prompt = file[start:start+seq_length]
print(prompt)
pattern = char_tensor(prompt)

#len(pattern[0])
pattern = pattern.tolist()



her, 


In [30]:
model_loaded = torch.load("LSTM_shakespear.pth")
print('Prompt: "%s"' % prompt)
with torch.no_grad():
    for i in range(100):
        x = np.reshape(pattern, (1, len(pattern), 1)) / float(n_characters)
        x = torch.tensor(x, dtype=torch.float32).to(device)
        prediction = model_loaded(x)
        index = int(prediction.argmax())
        result = int_to_char(index)
        print(result, end="")
        pattern.append(index)
        pattern = pattern[1:]


Prompt: "her, "
ems wf sh Ah hR Aeom wf hro 
Throes toomenos YTUOLWER IRUOhetnety te sooeesdRsyoR 
KLTAIWTEAWI 
KUIL

# Task 3

In [37]:
prompt1 = ['The ', 'What is ','Shall I give ','X087hNYB BHN BYFVuhsdbs ']


print('Prompts: %s' % prompt1)
for g in prompt1:
    pattern = char_tensor(g).tolist()
    print()
    print('Prompt: %s' % g)
    with torch.no_grad():
        for i in range(100):
            x = np.reshape(pattern, (1, len(pattern), 1)) / float(n_characters)
            x = torch.tensor(x, dtype=torch.float32).to(device)
            prediction = model_loaded(x)
            index = int(prediction.argmax())
            result = int_to_char(index)
            print(result, end="")
            pattern.append(index)
            pattern = pattern[1:]
    print()


Prompts: ['The ', 'What is ', 'Shall I give ', 'X087hNYB BHN BYFVuhsdbs ']

Prompt: The 
sf hI MTheehsws AR ArYaRdhsow WRIhrehy
Aldey Eh hI MNILUUUARAAA 
GLLhe sh th :ARAeT 
TRAeres Eh aRdt

Prompt: What is 
the theeyarc th Aou wios theee, I Ter I Tha the soodr,
Aod ta test Ahan thane youe to tear shale,
Ie

Prompt: Shall I give 
the shall then that the sea the shall the sears,
The was the throe the heart To heart that these
the

Prompt: X087hNYB BHN BYFVuhsdbs 
thot The sasett that the sher I then the hooorr of the shale the sears of the seats of the shaln of 
